In [6]:
# backend/utils.py
from collections import Counter
from nltk.tokenize import word_tokenize
import fitz  # PyMuPDF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

# Fonction pour extraire le texte d'un fichier PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as pdf_document:
        for page_num in range(len(pdf_document)):
            page = pdf_document.load_page(page_num)
            text += page.get_text()
    return text

# Fonction pour extraire les langages utilisés et les compétences à partir du texte du CV
def extract_languages_and_skills(cv_text):
    domain_keywords = {
        "skills": ["compétences", "techniques"],
        "certificates": ["certifications", "certificats"],
        "projects": ["personal", "projects"]  # Correction : "projects" au lieu de "PROJETS"
    }
    domain_info = {domain: {"languages": [], "skills": []} for domain in domain_keywords}
    tokens = word_tokenize(cv_text.lower())
    current_domain = ""
    for token in tokens:
        for domain, keywords in domain_keywords.items():
            if any(keyword in token for keyword in keywords):
                current_domain = domain
                break
        if current_domain != "":
            if token.isalpha():
                if token in ["html", "css", "php", "javascript", "java", "python", "c", "c++", "matlab"]:
                    domain_info[current_domain]["languages"].append(token)
                else:
                    domain_info[current_domain]["skills"].append(token)
        elif token in [":", "-", "—"]:
            current_domain = ""
    return domain_info

# Fonction pour extraire les compétences du CV
def extract_skills(cv_info):
    skills = []
    for domain, info in cv_info.items():
        skills.extend(info["skills"])
    return skills

# Fonction pour générer les questions pertinentes pour les compétences du CV
def generate_questions(cv_pdf_file, question_database_file, threshold=0.4):
    # Chargement du texte extrait du PDF
    pdf_text = extract_text_from_pdf(cv_pdf_file)
    # Extraction des langages utilisés et des compétences dans chaque domaine du CV
    cv_info = extract_languages_and_skills(pdf_text)
    # Suppression des doublons et conversion en liste
    question_database = pd.read_csv(question_database_file)
    questions_corpus = list(set(question_database["Question"]))
    # Création de la matrice TF-IDF
    vectorizer = TfidfVectorizer()
    questions_tfidf = vectorizer.fit_transform(questions_corpus)
    # Extraction des compétences
    skills = extract_skills(cv_info)
    skills_tfidf = vectorizer.transform(skills)
    # Calcul de la similarité cosinus entre les compétences et les questions
    similarity_scores = cosine_similarity(skills_tfidf, questions_tfidf)
    # Sélection des questions les plus pertinentes (avec une similarité élevée)
    relevant_questions_indices = np.where(similarity_scores > threshold)[1]
    relevant_questions = [questions_corpus[i] for i in relevant_questions_indices]
    return relevant_questions


In [7]:
from flask import Flask, request, jsonify
from flask_cors import CORS
 # Importez la fonction generate_questions depuis utils.py

app = Flask(__name__)
CORS(app)

@app.route("/generate_questions", methods=["POST"])
def generate_questions_route():
    # Assurez-vous d'envoyer le fichier PDF du CV dans le corps de la requête
    if 'cv_pdf_file' not in request.files:
        return 'No file part', 400
    cv_pdf_file = request.files["cv_pdf_file"]
    if cv_pdf_file.filename == '':
        return 'No selected file', 400

    question_database_file = "test_candidat_skills.csv"
    # Génération des questions pertinentes
    relevant_questions = generate_questions(cv_pdf_file, question_database_file)
    return jsonify(relevant_questions)

if __name__ == "__main__":
    app.run(debug=True, port=5000)  # Exécute l'application sur le port 8080


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\ASUS\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
